In [1]:
#https://github.com/kylebgorman/textgrid
#Autosegmenteerija poolt loodud TextGridide lihtsustamiseks
#!pip install textgrid

In [2]:
import pandas as pd
import numpy as np
import matplotlib
import matplotlib.pyplot as plt
from scipy.signal import butter, lfilter
import librosa
import librosa.display
import IPython.display as ipd
import soundfile as sf
import textgrid
import os
import errno
import glob
import os.path
import shutil
import subprocess, shlex
import multiprocessing
import time
import re
import math

#https://github.com/TartuNLP/tts_preprocess_et
from tts_preprocess_et.convert import convert_sentence
from multiprocessing import Process
from copy import copy

%matplotlib inline
matplotlib.rcParams['figure.figsize'] = [20, 15]

In [3]:
#MILLISEID EELTÖÖTLUS MEETODEID SOOVITE KASUTADA
preprocessUsingRMSZCR = False #Kas rakendada kõnekorpusele RMS + ZCR põhist vaikuse eemaldamist
                             #Bakalaureuse töös on kasutatud terminit RMSE, kuid librosa meetodi nimi on RMS (valem on sellele vaatamata sama)
preprocessUsingAutosegment = False #Kas rakendada kõnekorpusele Autosegmenteerija põhist eemaldamist

#KÕNEKORPUSTE JUURKAUST
originalAudioRootFolder = '/gpfs/hpc/projects/nlpgroup/speech-synthesis/data/original/'

#RAAMI JA DISKREETIMISSAGEDUSE SÄTTED
SR = 22050 #Diskreetimissagedus (sampling rate)
N_FFT = 512 #Raami suurus nii STFT kui ka RMS + ZCR puhul
HOP_LENGTH = 128 #Raami nihutus nii STFT kui ka RMS + ZCR puhul, seega kattuvus (overlap) on järgmise / eelmise raamiga on 75%

#AUTOSEGMENTEERIJA MUUTUJAD
autosegmentMaxSilence = 0.5 #Defineerib keskmiste vaikuste maksimaalse pikkuse sekundites
autosegmentAudioOutput = "Preprocessed-Autosegment/" #Väljundi kaust
textGridsFolder =  "TextGridsWithPreprocessedSents/" #Kaust, kus asuvad TextGridid

#LIHTSAKOELISE PUHTALT RMS MUUTUJAD
simpleTrimMaxSilence = 0.35 #Eeldades, et keskmised vaikused on leebelt jagatud, on mingi vaikus niikuinii sees, seega max on natuke suurem
simpleTrimMaxSilenceInFrames = int(simpleTrimMaxSilence*SR / HOP_LENGTH)
simpleTrimOutputFolderName = "Preprocessed-Simple-RMS/" #Väljundi kaust

#RMS + ZCR MEETODI MUUTUJAD
newTrimMaxSilence = 0.4 #Eeldades, et keskmised vaikused on leebelt jagatud, on mingi vaikus niikuinii sees
newTrimMaxSilenceInFrames = int(newTrimMaxSilence*SR / HOP_LENGTH)
newTrimOutputFolderName = "Preprocessed-RMS-ZCR/"

In [4]:
#Hääle lugejate dataframe kirjutamine

#Kuna iga kõnekorpus on erinev, siis on raske luua universaalset lahendust
#Seega kui Teie kõnekorpuse ülesehitus erineb drastiliselt  nt. UT uudiskorpusest, siis peate meetodit muutma

#Tähtis on koodiploki lõpuks luua Dataframe data, kus on vaikuse eemaldamise eeltöötlust soovivate failide nimekiri
#Vajalikud veerud on "speaker", kus on rääkija nimi, "filename", kus on helifaili asukoht "sentence", kus on faili loetud lause (töödeldud)

#NB! Lause all mõeldakse kogu helifailis etteloetud teksti. Kui helifailis loetakse ette mitu lauset, siis see loeb kui 1 lause.

#Kõnelejate nimed
UTspeakers = ['Mari', 'Kalev', 'Albert', 'Vesta']
EKIspeakers = ["Kylli","Meelis"]

preprocessSentences = True #Kas lauseid sisse lugedes rakendada lausete eeltöötlust (lühendid lahti kirjutada) (võib võtta aega (antud korpustel 15 minutit))


data = pd.DataFrame()
baseSentencesFiles = set() #Salvetame juurkausta, kus on korpuseid kirjeldavad csv failid. Kaustade nimekirjaga saab hiljem vajadusel csv faile ringi kopeerida

for speaker in UTspeakers:
    sentencesFile = originalAudioRootFolder + 'UT-uudised-' + speaker +  '/sentences.csv'
    sentences = pd.read_csv(sentencesFile, sep='|', header=None)
    sentences.columns = ['filename', 'uneditedSentence']
    sentences['speaker'] = speaker
    
    sentences["filename"] = sentences["filename"].map(lambda filename: "UT-uudised-" + speaker + '/' + filename)
    
    #https://stackoverflow.com/questions/55902042/python-keep-only-alphanumeric-and-space-and-ignore-non-ascii/55902074
    if preprocessSentences:
        sentences["sentence"] = sentences["uneditedSentence"].map(lambda sent: re.sub(r'[^A-Za-zšŠžŽöäüõÖÄÜÕ0-9 ]+', '', convert_sentence(sent)))
    else:
        sentences["sentence"] = sentences["uneditedSentence"].map(lambda sent: re.sub(r'[^A-Za-zšŠžŽöäüõÖÄÜÕ0-9 ]+', '', sent))
    

    
    data = data.append(sentences)
    
    sentencesFileRoot = originalAudioRootFolder + 'UT-uudised-' + speaker
    baseSentencesFiles.add(sentencesFileRoot)
    
    
for speaker in EKIspeakers:
    sentencesFile = originalAudioRootFolder + 'EKI-ilukirjandus-' + speaker +  '/sentences.csv'
    sentences = pd.read_csv(sentencesFile, sep='|', header=None)
    sentences.columns = ['filename', 'uneditedSentence']
    sentences['speaker'] = speaker
    
    sentences["filename"] = sentences["filename"].map(lambda filename: "EKI-ilukirjandus-" + speaker + '/' + filename)
    if preprocessSentences:
        sentences["sentence"] = sentences["uneditedSentence"].map(lambda sent: re.sub(r'[^A-Za-zšŠžŽöäüõÖÄÜÕ0-9 ]+', '', convert_sentence(sent)))
    else:
        sentences["sentence"] = sentences["uneditedSentence"].map(lambda sent: re.sub(r'[^A-Za-zšŠžŽöäüõÖÄÜÕ0-9 ]+', '', sent))
    
    data = data.append(sentences)
    
    sentencesFileRoot = originalAudioRootFolder + 'EKI-ilukirjandus-' + speaker
    baseSentencesFiles.add(sentencesFileRoot)

#Edasipidi tegeleme combinedData dataframega
#Notebooki lõpus olevate koodiblokkidega saab teha combinedDatat testimise eesmärgil väiksemaks, seega teeme igaks juhuks koopia
combinedData = data.copy()

In [5]:
#Kohustuslik abistav meetodit failide ja kaustade loomiseks

#Loob vajalikud kaustad nende puudumisel, et saaks luua faili outputPath
#https://stackoverflow.com/questions/273192/how-can-i-safely-create-a-nested-directory
def ifDirNotExistCreate(outputPath):
    if not os.path.exists(os.path.dirname(outputPath)):
        try:
            os.makedirs(os.path.dirname(outputPath))
        except OSError as exc: # Rallikonditsiooni vastane kaitse
            if exc.errno != errno.EEXIST:
                raise

In [6]:
# Kasutab autosegmenteerija tagastatud textgrid faili, et eemaldada failist vaikust
# Row on combinedData dataframe rida
def removeSilenceUsingAutoSegment(row):
    name = (".").join(row.filename.split(".")[0:-1]) #Kogu faili path v.a. tüüp (.wav)
    try:
        tg = textgrid.TextGrid.fromFile(textGridsFolder +name+'.TextGrid')
    except: #Kui textgrid pole loodud, nt. mingi karakter oli illegaalne
        print("Textgrid loading went wrong, tried to load "+ textGridsFolder +name+'.TextGrid')
        trimAndSave(row, False, autosegmentAudioOutput+name+".wav") #Kasutame oma loodud trimmerit
        return
    y, sr = librosa.load(name+".wav")
    y = librosa.util.normalize(y)
    filteredAudio = []
    for index, interval in enumerate(tg[0]):
        if interval.mark != "<sil>":
            filteredAudio.extend(y[librosa.core.time_to_samples(interval.minTime,sr):librosa.core.time_to_samples(interval.maxTime,sr)])
        else: #vaikus
            if (index == 0 or index == len(tg[0]) -1): #Kui tegemist on vaikusega ja see vaikus on esimene või viimane sektor, mida me kuuleme
                continue #Ei lisa leitud vaikust
            if(interval.maxTime - interval.minTime >= autosegmentMaxSilence): #Kui keskmine vaikus on liiga pikk, jätame vaikuse keskmise osa välja
                filteredAudio.extend(y[librosa.core.time_to_samples(interval.minTime,sr):
                                       librosa.core.time_to_samples(interval.minTime + (autosegmentMaxSilence / 2),sr)])
                filteredAudio.extend(y[librosa.core.time_to_samples(interval.maxTime - (autosegmentMaxSilence / 2),sr):
                                       librosa.core.time_to_samples(interval.maxTime,sr)])
            else: #piisavalt lühike, jätame kogu vaikuse sisse
                filteredAudio.extend(y[librosa.core.time_to_samples(interval.minTime,sr):librosa.core.time_to_samples(interval.maxTime,sr)])
    outputFileName =  autosegmentAudioOutput+name+".wav"
    ifDirNotExistCreate(outputFileName)
    sf.write(outputFileName, filteredAudio, sr)

In [7]:
#Eeldades, et igal helifailil on olemas sama nimega TextGrid fail, saame rakendada vaikuse eemaldajat

#Lõimede ärakasutamise eesmärgil meetod, mis võtab järjekorrast failinime ja töötleb vastavat faili (loob uue töödeldud faili)
def oneFileFromQueueToAutosegmentSilRemoval(taskQueue):
    while not taskQueue.empty():
        if(taskQueue.qsize() % 50 == 0):
            print("Files left to remove silence from:",taskQueue.qsize())
        rowWithIndex = taskQueue.get()
        removeSilenceUsingAutoSegment(rowWithIndex[0])
    return True

if preprocessUsingAutosegment: #Kui soovitakse kasutada Autosegmenteerijat
    start = time.time() #Kulunud aja mõõtmiseks

    taskQueue = multiprocessing.Queue()
    processes = 10 #Maksimaalne lõimede arv, 10 tundus optimaalne, liiga kõrge arv aeglustas
    proc = [] #Lõimelised protessid lähevad siia


    for index, row in combinedData.iterrows():
        name = (".").join(row.filename.split(".")[0:-1])
        taskQueue.put((row,index))
            
            
    for n in range(processes):
        p = Process(target=oneFileFromQueueToAutosegmentSilRemoval, args=(taskQueue,))
        proc.append(p)
        p.start()

    for p in proc:
        p.join()

    print("Time taken = ",time.time() - start, "s")

In [8]:
#RMS ja ZCR meetodi puhul keskmisi vaikusi trimmiv meetod
#states on list nullidest ja ühtedest, mis näitab, mis raamis on vaikus. 0 tähendab vaikust.
def applyMaxSilenceDuration(states, maxSilenceFrames):
    
    newStates = copy(states) #Teeme koopia, et põhimeetodis mitte mõjutada originaalset states olekut
                             #Sellesse koopiasse teeme muudatusi, et nt. lausete sisemist vaikust alles hoida
    startOfSilenceFrame = -1 #Jätame meelde, mis raamis algas viimati nähtus vaikus
    durationOfSilenceFrame = 0 #Praeguse vaikuse kestus raamides
    
    for index, val in enumerate(states):
        if val == 1: #Heli
            if durationOfSilenceFrame > maxSilenceFrames: #Enne heli oli vaikus, mis oli pikem kui lubatud
                newStates[startOfSilenceFrame : startOfSilenceFrame + int(maxSilenceFrames / 2) + 1] = 1
                newStates[index - int(maxSilenceFrames / 2) - 1 : index] = 1
            elif durationOfSilenceFrame > 0: #Enne heli oli vaikus, mis oli lubatu piires
                newStates[startOfSilenceFrame : index] = 1
            durationOfSilenceFrame = 0
            startOfSilenceFrame = -1
        else: #Vaikus
            durationOfSilenceFrame += 1
            if startOfSilenceFrame == -1:
                startOfSilenceFrame = index
    return newStates

In [9]:
#Uus loodav trimmija (kasutab ka ZCR ja arvestab heli alguses oleva vaikusega)
#Kui logida tulemusi, siis tehakse mitu joonist, nii et logimine on kasulik üksikute helifailide testimisel
def trimAndSave(row, log,outputName = None):
    
    if log:
        print(row.speaker + '/' + row.filename + '\n' + "Sent: "+ row.sentence)
    
    #Eeltöötlus
    audioFile = originalAudioRootFolder + row.filename
    transcript  = row.sentence #Ehk helifailis loetud lause
    
    y, _ = librosa.load(audioFile, sr=SR) #Laadime helifaili sisse diskreetimissagedusega SR
    yOriginal = y.copy() #Hiljem eemaldame vaikuse filtreerimata helisignaalist, seega vaja koopiat
    
    #Rakendame kõrg- ja madalpääs filtrit, et jätta alles vaid inimhääle sagedused
    #https://stackoverflow.com/questions/48101564/create-a-band-pass-filter-via-scipy-in-python
    lo,hi=250,6000
    b,a=butter(N=6, Wn=[2*lo/SR, 2*hi/SR], btype='band')
    yFiltered = lfilter(b,a,y)
    
    y = librosa.util.normalize(yFiltered)
    yOriginal = librosa.util.normalize(yOriginal)
    
    #RMS leidmine
    
    #Et lihtsalt rakendada aknafunktsiooni RMS leidmisel, teeme enne STFT (kus on automaatselt sees akna funktsioon (Hanni aken))
    #Samuti ei ole meil vaja STFT poolt antud faasi iga sageduskomponendi juures, ainult magnituude
    S, phase = librosa.magphase(librosa.stft(y, n_fft=N_FFT, hop_length=HOP_LENGTH))
    rms = librosa.feature.rms(S=S, frame_length=N_FFT, hop_length=HOP_LENGTH) [0]
    
    #Lihtsustatud versiooni kasutamise otsustamine
    rmsNoiseThresholdBeginning = np.average(rms[17:34]) #Esimesed 0.1 - 0.2 sekundid helifailist
    
    if(rmsNoiseThresholdBeginning / max(rms) > 0.1): #Kui juba helifaili alguses algab mitte vaikus, ei saa tulevasi meetodeid rakendada
        print("Soundfile starts with speech "+ row.speaker + '/' + row.filename)
        trimAndSaveSimple(row, log, newTrimOutputFolderName+row.filename) #Kasutame oma varem loodud trimmerit
        return
    
    #Läve leidmine
    rmsNoiseThreshold = 0.06 * (max(rms) - np.average(rms[17:34]))
    
    #Viterbi kasutamine leebe ja range märgenduse loomiseks
    #1 - Range vaikuse märgendaja äärmiste vaikuste jaoks
    #2 - Leebem vaikuse märgendaja keskmiste vaikuste jaoks
    
    
    # https://librosa.org/doc/main/auto_examples/plot_viterbi.html#sphx-glr-auto-examples-plot-viterbi-py
    times = librosa.times_like(rms, sr=SR, hop_length=HOP_LENGTH)
    
    #Leiame RMS erinevuse läve ja vaadeldava raami vahel
    #Siis laiendame/normaliseerime väärtuste vahemikku, kuna rms väärtused on tavaliselt väiksed ([0,0.3])
    rNormalized1 = (rms - 1.2*rmsNoiseThreshold) / np.std(rms) #1.2*rmsNoiseThreshold on RMS väärtus, mida saab kõneks lugeda
    rNormalized2 = (rms - 1.1*rmsNoiseThreshold) / np.std(rms)
    
    
    p1 = np.exp(rNormalized1) / (1 + np.exp(rNormalized1))
    p2= np.exp(rNormalized2) / (1 + np.exp(rNormalized2))

    
    transition1 = librosa.sequence.transition_loop(2, [0.5, 0.58])
    transition2 = librosa.sequence.transition_loop(2, [0.5, 0.6]) # Kui enne oli heli, eeldame, et pigem tuleb uuesti heli

    
    fullP1 = np.vstack([1 - p1, p1])
    fullP2 = np.vstack([1 - p2, p2])

    states1 = librosa.sequence.viterbi_discriminative(fullP1, transition1)
    states2 = librosa.sequence.viterbi_discriminative(fullP2, transition2)
    
    
    if log:
        fig, ax = plt.subplots()
        ax.set_title(row.filename)
        ax.plot(times, p1, label='P[V=1|x]')
        ax.axhline(0.5, color='r', alpha=0.5, label='Descision threshold')
        ax.set(xlabel='Time')
        ax.legend();

    #Range märgenduse töötlemine ja kõne otspunktide leidmine
    firstStrictSilenceFound = False # Kas oleme leidnud esimese vaikuse
    lastFittingIndex = len(states1) - 1 #Salvestame siia viimase mittevaikuse raami
    lastFittingIndexFrameCounter=0 #Mitu raami on viimati leitud mittevaikus kestnud? (Liiga väiksed helid ei loe)
    firstFittingIndex = 0 #Salvestame siia esimese mittevaikuse raami
    for index, state in enumerate(states1): # Range heli tunnetaja
        if  state == 0: #Kui vaikus
            lastFittingIndexFrameCounter = 0
            if not firstStrictSilenceFound: #Kuna helifail võib alata mehhaanilise müraga, enne kõne märkimist võiks oodata esimese vaikuseni
                firstStrictSilenceFound = True
                
        #Kui esimene vaikus on leitud ja tegemist on esimese mitte vaikusega
        #Samuti on tegu kõnega ja kõne on kestnud vähemalt 12 raami (0,07 s)
        if  (firstStrictSilenceFound and firstFittingIndex == 0 and 
            state == 1 and lastFittingIndexFrameCounter > 12):
            firstFittingIndex = index - lastFittingIndexFrameCounter
        if  state == 1: #Kui kõne
            lastFittingIndexFrameCounter += 1
            if lastFittingIndexFrameCounter > 1: #Ja kõne on kestnud vähemalt 1 raami ehk 0.023 s
                lastFittingIndex = index
        
    #Leebe märgenduse töötlemine
    states2MaxSilenced = applyMaxSilenceDuration(states2, newTrimMaxSilenceInFrames)
    

    #zero crossing rate leidmine ja läve arvutamine
    zeroCR = librosa.feature.zero_crossing_rate(y, frame_length=N_FFT, hop_length=HOP_LENGTH)[0]
    ZCRThreshold = np.average(zeroCR[17:34]) + 2.1* np.std(zeroCR[17:34])
    
    #ZCR ajavahemiku arvutamine
    newTrimBaseZCRBonusWindowDuration = 0.12 #sekundites
    
    if len({"g","b","d","k","p","t","f","h"} - set(transcript[-2:])) != 8 and transcript[-1] not in ["a","e","i","o","u","õ","ä","ö","ü"]:
        newTrimBaseZCRBonusWindowDuration += 0.04
    if(transcript[-2:] in ["kk","pp","tt","ll"]):
        newTrimBaseZCRBonusWindowDuration += 0.04
    
    newTrimZCRBonusWindowInFrames = int(newTrimBaseZCRBonusWindowDuration*SR / HOP_LENGTH)
    
    #Kõne otspunktide täpsustamine
    firstFittingIndex = max(firstFittingIndex - 12,0)
    
    updatedLastFittingIndex = lastFittingIndex
    for ind, zcr in enumerate(zeroCR[lastFittingIndex:lastFittingIndex + newTrimZCRBonusWindowInFrames +1], start=lastFittingIndex):
        if zcr > ZCRThreshold:
            updatedLastFittingIndex = ind
        

    updatedLastFittingIndex = min(updatedLastFittingIndex + 9,len(states2))

    #Lõpliku märgenduse loomine
    states2MaxSilenced[0:firstFittingIndex] = 0
    states2MaxSilenced[lastFittingIndex:updatedLastFittingIndex] = 1
    states2MaxSilenced[updatedLastFittingIndex:] = 0
    
    
    if log:
        fig, ax = plt.subplots(nrows=1, sharex=True)
        #ax.set_title(row.filename + "\n" + row.sentence)
        ax.set_title(row.filename)
        ax.label_outer()
        
        #ax.step(times, zeroCR, linestyle='--', color='pink', label='ZCR')
        librosa.display.waveplot(y,alpha=0.5)
        #ax.step(times, states2, linestyle='--', color='green', label='ViterbiForgiving')
        ax.step(times, states1, linestyle='--', color='red', label='ViterbiStrict')
        reversedScale = [-x for x in states2MaxSilenced]
        ax.step(times, reversedScale, linestyle='--', color='black', label='ViterbiFinal')
        ax.set(ylim=[-1.1, 1.1])
        ax.legend(loc="upper right")


        
    if log:
        plt.figure(figsize=(25, 20))
        librosa.display.waveplot(y,alpha=0.5)
        plt.title("Final Viterby with ZCR and threshold")
        plt.hlines(ZCRThreshold,xmin=0,xmax=librosa.get_duration(yFiltered), colors=["black"])
        plt.plot(times, zeroCR, color="r", label = "ZCR")
        plt.plot(times, states2MaxSilenced, linestyle='--', color='black', label='ViterbiFinal')
        plt.legend()
        plt.show()

    
    #Siia loome uue helifaili
    newSound = []
    
    #Käime uuendatud üldise helimärgenduste listi läbi ja lisame ainult need heliosad, kus oleme heli märgendanud
    for index,state in  enumerate(states2MaxSilenced):
        if(state == 1):
            newSound.extend(yOriginal[index*HOP_LENGTH:((index+1)*HOP_LENGTH)])

    #Loome kausta kuhu panna failid ja kirjutame need sinna
    outputPath = newTrimOutputFolderName+row.filename
    if outputName != None:
        outputPath = outputName
    ifDirNotExistCreate(outputPath)
    sf.write(outputPath, newSound, SR)
    return


In [10]:
#Uus loodav trimmija (ainult RMS põhine ja lävi ei arvesta helifaili alguses oleva vaikusega)
def trimAndSaveSimple(row, log,outputName = None):
    if log:
        print(row.speaker + '/' + row.filename + '\n' + "Sent: "+ row.sentence)
    
    #Eeltöötlus
    audioFile = originalAudioRootFolder + row.filename
    transcript  = row.sentence
    
    
    y, _ = librosa.load(audioFile, sr=SR)
    y = librosa.util.normalize(y)

    #RMS leidmine
    S, phase = librosa.magphase(librosa.stft(y, n_fft=N_FFT, hop_length=HOP_LENGTH))
    rms = librosa.feature.rms(S=S, frame_length=N_FFT, hop_length=HOP_LENGTH)  [0]
    
    
    #Arvestame läve arvutamisel kõneleja keskkonna ja kõneleja valjusega
    shouldBeAvg = 0.085 # => Kui helifaili keskmine RMS on 0.085, ei muudeta läve kordjat
    actualAvg = rms[rms > 0.01].mean()
    multiplier = 1 + (actualAvg - shouldBeAvg) * 8
    
    #Viterbi kasutamine range ja leebe märgendamise jaoks
    # https://librosa.org/doc/main/auto_examples/plot_viterbi.html#sphx-glr-auto-examples-plot-viterbi-py
    times = librosa.times_like(rms, sr=SR, hop_length=HOP_LENGTH)
    
    
    rNormalized1 = (rms - 0.012*multiplier) / np.std(rms)
    rNormalized2 = (rms - 0.011*multiplier) / np.std(rms)
    p1 = np.exp(rNormalized1) / (1 + np.exp(rNormalized1))
    p2 = np.exp(rNormalized2) / (1 + np.exp(rNormalized2))

    
    transition1 = librosa.sequence.transition_loop(2, [0.5, 0.50]) # Ignoreerime minevikku faase
    transition2 = librosa.sequence.transition_loop(2, [0.5, 0.58]) # Kui enne oli heli, eeldame, et pigem tuleb uuesti heli

    
    fullP1 = np.vstack([1 - p1, p1])
    fullP2 = np.vstack([1 - p2, p2])

    states1 = librosa.sequence.viterbi_discriminative(fullP1, transition1)
    states2 = librosa.sequence.viterbi_discriminative(fullP2, transition2)
    
    
    if log:
        fig, ax = plt.subplots()
        ax.set_title(row.filename)
        ax.plot(times, p1, label='P[V=1|x]')
        ax.axhline(0.5, color='r', alpha=0.5, label='Descision threshold')
        ax.set(xlabel='Time')
        ax.legend();

    #Range märgendamise töötlemine ja kõne otspunktide leidmine
    firstStrictSilenceFound = False # Kas oleme leidnud esimese vaikuse
    lastFittingIndex = len(states1) - 1 #Salvestame siia viimase mittevaikuse raami
    lastFittingIndexFrameCounter=0 #Mitu raami on viimati leitud vaikus kestnud? (Liiga väiksed helid ei loe)
    firstFittingIndex = 0 #Salvestame siia esimese mittevaikuse raami
    for index, state in enumerate(states1): # Range heli tunnetaja
        if  state == 0: #Kui vaikus
            lastFittingIndexFrameCounter = 0
            if not firstStrictSilenceFound: #If we have not yet found first silence for the trimmer (to  ignore instant  noise)
                firstStrictSilenceFound = True
        if  firstStrictSilenceFound and firstFittingIndex == 0 and state == 1 and lastFittingIndexFrameCounter > 12: #Kui esimene vaikus on leitud ja tegemist on esimese mitte vaikusega
            firstFittingIndex = index - lastFittingIndexFrameCounter
        if  state == 1: #Kui kõne
            lastFittingIndexFrameCounter += 1
            if lastFittingIndexFrameCounter > 12:
                lastFittingIndex = index
           
        
    #Leebe märgendamise töötlemine
    states2MaxSilenced = applyMaxSilenceDuration(states2, simpleTrimMaxSilenceInFrames)
    
    #Transkriptsiooniga arvestamine (helitud häälikud)
    bonusFrames = 36 #Lause lõpus jätame nii palju diskreete alles ka pärast vaikuse tundmist (et kompenseerida agressiivsust) [0.21 sekundit]
    if len({"g","b","d","k","p","t","f","h"} - set(transcript[-2:])) != 8 and transcript[-1] not in ["a","e","i","o","u","õ","ä","ö","ü"]:
        bonusFrames += 24 #0.14 sekundit
    if(transcript[-2:] in ["kk","pp","tt","ll"]):
        bonusFrames += 20 #0.12 sekundit
    
    
    #Kõne otspunktide parandamine
    firstFittingIndex = max(firstFittingIndex - 15,0)
    lastFittingIndex = min(lastFittingIndex + bonusFrames,len(states2))
    
    #Lisame leebele vaikuse leidajale vajalikud märkmed kõne otspunktidest
    states2MaxSilenced[0:firstFittingIndex] = 0
    states2MaxSilenced[lastFittingIndex:] = 0
    
    
    if log:
        fig, ax = plt.subplots(nrows=1, sharex=True)
        ax.set_title(row.filename)
        librosa.display.waveplot(y,alpha=0.5)
        ax.label_outer()
        
        #ax.step(times, p2>=0.5, label='Frame-wise')
        #ax.step(times, states2, linestyle='--', color='green', label='ViterbiForgiving')
        ax.step(times, states1, linestyle='--', color='red', label='ViterbiStrict')
        reversedScale = [-x for x in states2MaxSilenced]
        ax.step(times, reversedScale, linestyle='--', color='black', label='ViterbiFinal')
        ax.set(ylim=[-1.1, 1.1])
        ax.legend()

    
    
    

    
    #Siia loome uue helifaili
    newSound = []
    
    #Käime lõpliku helimärgenduste listi läbi ja lisame ainult need heliosad, kus oleme heli märgendanud
    for index,state in  enumerate(states2MaxSilenced):
        if(state == 1):
            #Tegelikult peaks lisama natuke väärtust diskreetide indeksitele, et need oleksid raami keskel, mitte alguses
            newSound.extend(y[index*HOP_LENGTH:((index+1)*HOP_LENGTH)])

    #Loome kausta kuhu panna failid ja kirjutame need sinna
    outputPath = simpleTrimOutputFolderName+row.filename
    if outputName != None:
        outputPath = outputName
        
    ifDirNotExistCreate(outputPath)
    sf.write(outputPath, newSound, SR)
    return


In [11]:
def oneFileFromQueueToRMS_ZCRSilRemoval(taskQueue):
    while not taskQueue.empty():
        if(taskQueue.qsize() % 10 == 0):
            print("Files left to remove silence from:",taskQueue.qsize())
        rowWithIndex = taskQueue.get()
        trimAndSave(rowWithIndex[0], False)
    return True


if preprocessUsingRMSZCR: #Kui soovitakse kasutada RMS + ZCR meetodit eeltöötluseks
    start = time.time()


    taskQueue = multiprocessing.Queue()
    processes = 10
    proc = []


    for index, row in combinedData.iterrows():
        taskQueue.put((row,index))

    for n in range(processes):
        p = Process(target=oneFileFromQueueToRMS_ZCRSilRemoval, args=(taskQueue,))
        proc.append(p)
        p.start()

    for p in proc:
        p.join()

    print("Time taken = ",time.time() - start, "s")

In [12]:
#Lisa tööriistad, nt. failide kopeerimiseks, combinedData muutmiseks või informatsiooni saamiseks helifailidest

In [13]:
#Mitte kohustuslikud abistavad meetodid, kui on vaja faile kopeerida kaustast kausta, näiteks kodukausta kopeerimiseks / teise serverisse kopeerimiseks


def writeSentenceCSVFilesFromFolderToFolder(originFolder,destinationFolder):
    listOfCSVFiles =  glob.glob(originFolder +"/*.csv")
    for csvFile in listOfCSVFiles:
        ifDirNotExistCreate(destinationFolder)
        shutil.copy(csvFile,destinationFolder)
        
def writeSentenceOfAudioFileToFolder(folderName,filename,sentence):
    ifDirNotExistCreate(folderName+"/"+filename)
    with open(folderName+"/"+ ((".").join(filename.split(".")[0:-1])) + ".txt", 'w') as writer:
        writer.write(sentence)
        
#loob kodukaustas vajalikud kaustad, kopeerib helifailid kodukausta, kirjutab samasse kausta eraldi tekstifaili igale lausele
#Tagastab tegeletud kaustade hulga
def copyAudioFromOriginalFolderAndWriteSentenceToSeperateFile(data):
    for index, row in data.iterrows():
        original_audio_file_path = originalAudioRootFolder + row.filename

        #Teeme kindalks, et kodukaustas vajalikud kaustad eksisteerivad ja kui ei ekisteeri, siis loome'
        #https://stackoverflow.com/questions/12517451/automatically-creating-directories-with-file-output
        filename = row.filename
        foldername = ("/").join(filename.split("/")[0:-1]) #Ignoreerib faili nime osa (nt. UT-uudised-Albert)
        ifDirNotExistCreate(filename)
        shutil.copy(original_audio_file_path,foldername)
        with open((".").join(filename.split(".")[0:-1]) + ".txt", 'w') as writer:
            writer.write(row.sentence)

In [14]:
#Kui on soov sättida valikuks kõik data kirjed

# combinedData = data.copy()

In [15]:
#Kui on soov sättida valikuks osa data kirjetest, ning see valik teha suvaliselt
#from sklearn.utils import shuffle

# limit = 100
# VestaData= shuffle(data[data["speaker"]=="Vesta"])
# VestaData.reset_index(inplace=True, drop=True)
# VestaData= VestaData[0:limit]

# AlbertData= shuffle(data[data["speaker"]=="Albert"])
# AlbertData.reset_index(inplace=True, drop=True)
# AlbertData= AlbertData[0:limit]

# KalevData= shuffle(data[data["speaker"]=="Kalev"])
# KalevData.reset_index(inplace=True, drop=True)
# KalevData= KalevData[0:limit]

# MariData= shuffle(data[data["speaker"]=="Mari"])
# MariData.reset_index(inplace=True, drop=True)
# MariData= MariData[0:limit]

# KylliData= shuffle(data[data["speaker"]=="Kylli"])
# KylliData.reset_index(inplace=True, drop=True)
# KylliData= KylliData[0:limit]

# MeelisData= shuffle(data[data["speaker"]=="Meelis"])
# MeelisData.reset_index(inplace=True, drop=True)
# MeelisData= MeelisData[0:limit]

# combinedData = VestaData.append([AlbertData,KalevData,MariData,MeelisData,KylliData])

In [16]:
#Kui on soov sättida valikuks osa data kirjetest, ning see valik oleks esimesed limit kirjet

# limit = 15
# VestaData= (data[data["speaker"]=="Vesta"])
# VestaData= VestaData[0:limit]

# AlbertData= (data[data["speaker"]=="Albert"])
# AlbertData= AlbertData[0:limit]

# KalevData= (data[data["speaker"]=="Kalev"])
# KalevData= KalevData[0:limit]

# MariData= (data[data["speaker"]=="Mari"])
# MariData= MariData[0:limit]

# KylliData= (data[data["speaker"]=="Kylli"])
# KylliData= KylliData[0:limit]

# MeelisData= (data[data["speaker"]=="Meelis"])
# MeelisData= MeelisData[0:limit]

# combinedData = VestaData.append([AlbertData,KalevData,MariData,KylliData,MeelisData])

In [17]:
#Kirjutab ühe kausta csv failid teise kausta

# destinationBase = './SentsForERR/'

# for origin in baseSentencesFiles:
#     writeSentenceCSVFilesFromFolderToFolder(origin,destinationBase + origin.split("/")[-1] + "/")

In [18]:
#Kirjutab iga rea kohta datas lause faili

# for index, row in combinedData.iterrows():
#     writeSentenceOfAudioFileToFolder("ERRSentencesWithoutPreprocessingTextFilesUpdated",row.filename,row.sentence)

In [19]:
#Esiteks kopeeritakse vajalikud helifailid kodukausta ja luuakse igale helifailile eraldi tekstifail lausega (kirjutatavaks lauseks on datas olev sentence väärtus)

# copyAudioFromOriginalFolderAndWriteSentenceToSeperateFile(combinedData)

In [21]:
#Oli kasutusel et leida, kas pärast helifailide normaliseerimist on RMS väärtus sarane, sõltumata lugejast
#Selgus, et ikkagi mingi vahe on, seega on mõtekas tõsta RMS läve, kui üldiselt on RMS väärtus helifailis kõrge
def findAvgRMS(data):
    counter = 0
    for index, row in data.iterrows():
        audio_file = originalAudioRootFolder + row.filename
        y, samplerate = librosa.load(audio_file)
        y = librosa.util.normalize(y)
        S, phase = librosa.magphase(librosa.stft(y, n_fft=N_FFT, hop_length=HOP_LENGTH))
        rms = librosa.feature.rms(S=S, frame_length=N_FFT, hop_length=HOP_LENGTH)  [0]
        rms = rms[rms > 0.01]
        counter = counter + rms.mean()
        print(index," -> ",rms.mean())
    return counter / data.shape[0]
  
# AvgRMSVesta = findAvgRMS(VestaData[0:300])  
# AvgRMSAlbert = findAvgRMS(AlbertData[0:300])  
# AvgRMSKalev = findAvgRMS(KalevData[0:300])  
# AvgRMSMari = findAvgRMS(MariData[0:300])  

#print("Vesta -> ",AvgRMSVesta) #0.08319149660553583
#print("Albert -> ",AvgRMSAlbert)#0.07957464025042947
#print("Kalev -> ",AvgRMSKalev)#0.08875116641977554
#print("Mari -> ",AvgRMSMari)#0.0805271124581126